In [1]:
from ipynb.fs.full.Observer import CaptureConfiguration
import matplotlib.pyplot as plt
import cv2
from flask import Flask, Response, request
import json
from datetime import datetime, timedelta

In [2]:
print("Building Harmony Machine")
cc = CaptureConfiguration()

Building Harmony Machine


In [3]:
app = Flask(__name__)


captureInProgress = False
lastCaptureTime = None
annotations = []


def genDiceCam():
    global captures, lastCaptureTime
    while True:
        orig = cc.capture()['Dice']
        camImage = orig.copy()
        cv2.putText(camImage, f'Capturing: {captureInProgress}',
            (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 3, cv2.LINE_AA)
        annotationStr = ""
        for a in annotations:
            coords = json.loads(a['coords'])
            sp = [int(d) for d in coords['p0']]
            ep = [int(d) for d in coords['p1']]
            if captureInProgress:
                value = a['value']
                w, h = [d1 - d0 for d1, d0 in zip(ep, sp)]
                annotationStr += f"{value} {sp[0]} {sp[1]} {w} {h}\n"
            cv2.rectangle(camImage, sp, ep, (0, 255, 100), 3)
        ret, camImage = cv2.imencode('.jpg', camImage)

        if captureInProgress and (lastCaptureTime is None or (datetime.utcnow() - lastCaptureTime).total_seconds() > 30):
            imageName = f"diceCollector_Cap{datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')}"
            cv2.imwrite(f"captures/{imageName}.jpg", orig)
            with open(f"captures/{imageName}.txt", "w") as f:
                f.write(annotationStr)
            lastCaptureTime = datetime.utcnow()
            
        yield (b'--frame\r\n'
               b'Content-Type: image/jpg\r\n\r\n' + camImage.tobytes() + b'\r\n')


@app.route('/cam', methods=['GET'])
def getCam():
    return Response(genDiceCam(), mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/htmx.min.js', methods=['GET'])
def getHTMX():
    with open("templates/htmx.min.js", "r") as f:
        page = f.read()
    return Response(page, mimetype="application/javascript")


@app.route('/bootstrap.min.js', methods=['GET'])
def getBootstrapJS():
    with open("templates/bootstrap.min.js", "r") as f:
        page = f.read()
    return Response(page, mimetype="application/javascript")


@app.route('/bootstrap.min.css', methods=['GET'])
def getBootstrapCSS():
    with open("templates/bootstrap.min.css", "r") as f:
        page = f.read()
    return Response(page, mimetype="text/css")


@app.route('/console', methods=['GET'])
def getConsole():
    with open("templates/diceCollector.html", "r") as f:
        page = f.read()
    return Response(page, mimetype="text/html")


@app.route('/annotations', methods=['POST'])
def setAnnotations():
    global annotations
    annotations = json.loads(request.get_data(as_text=True))
    return Response("Success!", mimetype="text/html")


@app.route('/startcapture', methods=['POST'])
def startCapture():
    global captureInProgress, annotations
    captureInProgress = True
    annotations = json.loads(request.get_data(as_text=True))
    return Response("Success!", mimetype="text/html")


@app.route('/clearannotations', methods=['GET'])
def clearAnnotations():
    global captureInProgress, annotations
    captureInProgress = False
    annotations = []
    return Response("Success!", mimetype="text/html")

In [ ]:
if __name__ == "__main__":
    PORT = 5000
    print(f"Launching Server on {PORT}")
    app.run(host="0.0.0.0", port=PORT)

Launching Server on 5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.131:5000
Press CTRL+C to quit
192.168.2.102 - - [19/Oct/2023 11:01:06] "GET /console HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:06] "GET /bootstrap.min.css HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:06] "GET /bootstrap.min.js HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:06] "GET /htmx.min.js HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:07] "GET /cam HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:07] "GET /favicon.ico HTTP/1.1" 404 -
192.168.2.102 - - [19/Oct/2023 11:01:18] "POST /annotations HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:24] "POST /annotations HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:30] "POST /annotations HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:01:35] "POST /startcapture HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 11:15:46] "GET /clearannotations HTTP/1.1" 200 -
192.168.2.102 - - [19/Oct/2023 